### Install and import dependencies

In [ ]:
%pip install segment-geospatial leafmap localtileserver

In [ ]:
import requests
import os
import leafmap
import torch
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

### Download map

In [ ]:
image = 'http://umbra-open-data-catalog.s3.amazonaws.com/sar-data/tasks/Panama%20Canal%2C%20Panama/19812a4f-bc58-4109-abe8-cd7aa422cfe8/2023-02-19-15-08-26_UMBRA-04/2023-02-19-15-08-26_UMBRA-04_GEC.tif'

In [ ]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source='Satellite')

In [ ]:
m.add_raster(image, layer_name='Image')
m

### Initialize SAM class

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
checkpoint = os.path.join(out_dir, 'sam_vit_h_4b8939.pth')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam = SamGeo(
    checkpoint=checkpoint,
    model_type='vit_h',
    device=device,
    erosion_kernel=(3, 3),
    mask_multiplier=255,
    sam_kwargs=None,
)

### Segment image

In [ ]:
mask = 'segment.tiff'
sam.generate(image, mask)

### Polygonize raster data and save results to GeoPackage file

In [ ]:
vector = 'segment.gpkg'
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

In [ ]:
shapefile = 'segment.shp'
sam.tiff_to_vector(mask, shapefile)

### Visualize results

In [ ]:
style = {
    'color': '#3388ff',
    'weight': 2,
    'fillColor': '#7c4185',
    'fillOpacity': 0.5,
}
m.add_vector(vector, layer_name='Vector', style=style)
m